**PENDIENTE DE VERIFICAR SI ES NECESARIO**

Comprobación de PyTorch y uso de la GPU

In [ ]:
import torch

print(f"Versión de PyTorch: {torch.__version__}")
print(f"Versión de CUDA en PyTorch: {torch.version.cuda}")
print(f"¿CUDA está disponible?: {torch.cuda.is_available()}")
print(torch.cuda.get_device_name(0))

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11n-seg.pt")

results = model.train(data="./datasets/basketball_court_zones_segmentation_v2/data.yaml", imgsz=640, batch=8, device=0, epochs=60, task="segment")

Código reutilizado de BCT

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO  #Necesario para detectar con YOLO

#Intersección enter dos líneas
def line_intersection(line1, line2):
    x1, y1, x2, y2 = line1
    x3, y3, x4, y4 = line2

    # Calcular  pendientes e intersecciones
    denom = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
    if denom == 0:  # Líneas paralelas o coincidentes
        return None

    intersect_x = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / denom
    intersect_y = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / denom

    return (intersect_x, intersect_y)

#Comprueba que el punto esté dentro de la imagen
def is_within_image(point, image_width, image_height):
    x, y = point
    return 0 <= x < image_width and 0 <= y < image_height


# Cargar el modelo YOLO entrenado
model = YOLO("./runs/segment/train/weights/best.pt")

video_path = "../clips/Clip1LF.mp4"
# Cargar el video de entrada
cap = cv2.VideoCapture(video_path)

# Obtener detalles del video
fourcc = cv2.VideoWriter_fourcc(*"avc1")  # Codec para el video de salida
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Realizar detección en el frame
    results = model(frame)
    # Dimensiones del frame original
    frame_height, frame_width = frame.shape[:2]

    # Iterar sobre los resultados
    for result in results:
        masks = result.masks.data.cpu().numpy()  # Máscaras (NumPy array)
        classes = result.boxes.cls.cpu().numpy()  # Clases detectadas
        boxes = result.boxes.xyxy.cpu().numpy()  # Bounding boxes
        image = frame.copy()  # Crear copia del frame original para visualización

        # Aplicar y mostrar máscaras
        for i, cls in enumerate(classes):
            if model.names[cls] == 'paint':
                # Redimensionar la máscara al tamaño del frame original
                zone_mask = (masks[i] * 255).astype(np.uint8)  # Escalar la máscara (0-255)
                resized_zone_mask = cv2.resize(zone_mask, (frame_width, frame_height), interpolation=cv2.INTER_NEAREST)

                # Superponer la máscara en el frame original
                image[resized_zone_mask > 0] = (0, 0, 255)  # Rojo


    # Mostrar la zona en pantalla
    cv2.imshow("La zona", image)
    # Esperar por una tecla: espacio para avanzar, 'q' para salir
    key = cv2.waitKey(0) & 0xFF  # Espera indefinidamente hasta que se presione una tecla
    if key == ord('q'):
        break

# Liberar recursos
cap.release()

cv2.destroyAllWindows()

# Esperar a que se presione una tecla y cerrar ventana
cv2.waitKey(0)

cv2.destroyAllWindows()

Entrenamiento de SAM 2

In [ ]:
import os
import json
import cv2
from pycocotools import mask as maskUtils

# Ruta a las carpetas del dataset
dataset_dir = './datasets/sam2_basketball_court_zones_segmentation_v2'
phases = ['train', 'valid', 'test']

# Función para decodificar máscaras RLE
def decode_rle(segmentation, size):
    """Convierte una máscara RLE en una máscara binaria."""
    rle = {
        'counts': segmentation['counts'].encode('utf-8'),  # Asegúrate de codificar en bytes
        'size': size
    }
    mask = maskUtils.decode(rle)
    return mask

# Función para cargar datos con imágenes y máscaras
def load_dataset(phase):
    data = []
    folder = os.path.join(dataset_dir, phase)
    
    for file in os.listdir(folder):
        if file.endswith('.jpg') or file.endswith('.png'):
            img_path = os.path.join(folder, file)
            ann_path = os.path.join(folder, file.rsplit('.', 1)[0] + '.json')  # Cambia la extensión
            
            # Cargar imagen
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Cargar anotaciones
            with open(ann_path, 'r') as f:
                annotations = json.load(f)
            
            masks = []
            for annotation in annotations.get('annotations', []):
                segmentation = annotation.get('segmentation')
                size = segmentation.get('size', [image.shape[0], image.shape[1]])

                if segmentation and 'counts' in segmentation:
                    mask = decode_rle(segmentation, size)
                    masks.append(mask)

            data.append({
                'image': image,
                'masks': masks,
                'annotations': annotations
            })
    return data

# Cargar datos
train_data = load_dataset('train')
valid_data = load_dataset('valid')
test_data = load_dataset('test')

# Verificación de los datos cargados
print(f"Número de imágenes de entrenamiento: {len(train_data)}")
print(f"Número de imágenes de validación: {len(valid_data)}")
print(f"Número de imágenes de prueba: {len(test_data)}")

# Verificar las máscaras de la primera imagen
if train_data:
    print(f"Número de máscaras en la primera imagen: {len(train_data[0]['masks'])}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pycocotools.mask import decode
import random

# Ruta al dataset
dataset_dir = './datasets/sam2_basketball_court_zones_segmentation_v2'
phase = 'train'  # Puedes cambiar a 'valid' o 'test'
image_file = 'vid_1frame_3_jpg.rf.3bc44ec6354b8189ebafa57d71fda1f7.jpg'  # La primera imagen del JSON

# Cargar la imagen
image_path = os.path.join(dataset_dir, phase, image_file)
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convertir de BGR a RGB

# Cargar anotaciones
json_path = image_path.replace('.jpg', '.json')
with open(json_path, 'r') as f:
    data = json.load(f)

# Dibujar la imagen original
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.axis("off")

# Decodificar y dibujar cada máscara con un color diferente
for ann in data['annotations']:
    rle = ann['segmentation']
    mask = decode(rle)  # Decodificar la máscara RLE

    # Generar un color aleatorio para cada máscara
    color = [random.randint(0, 255) for _ in range(3)]
    color_mask = np.zeros_like(image, dtype=np.uint8)
    for i in range(3):  # R, G, B
        color_mask[:, :, i] = mask * color[i]

    # Superponer con transparencia
    blended = cv2.addWeighted(image, 0.5, color_mask, 0.5, 0)

    # Mostrar la imagen con la máscara de color
    plt.imshow(blended, alpha=0.5)

plt.title("Imagen con máscaras de diferentes colores")
plt.show()

In [ ]:
from transformers import SamModel, SamConfig

# Load the pre-trained SAM model
model = SamModel.from_pretrained("facebook/sam-vit-large")

In [ ]:
from torch.utils.data import Dataset
import torch

class BasketballCourtDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        image = item['image']
        masks = item['masks']
        
        # Convert masks to a single tensor
        mask = torch.stack([torch.tensor(m, dtype=torch.float32) for m in masks])
        
        if self.transform:
            image = self.transform(image)
        
        return {
            'image': image,
            'masks': mask
        }

In [ ]:
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

# Define the dataset and dataloader
train_dataset = BasketballCourtDataset(train_data)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        images = batch['image']
        masks = batch['masks']
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, masks)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')